In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.autonotebook import tqdm

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

/tmp/ipykernel_3141050/2386628013.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
pillar_df = pd.read_csv("/data/dzeiberg/pillar_project/dataframe/pillar_data_clinvar38_annotated_final_v13_expanded_031425.csv")

/tmp/ipykernel_3141050/807609826.py:1: DtypeWarning: Columns (4,11,13,14,15,21,22,23,34,37,38,39,40,43) have mixed types. Specify dtype option on import or set low_memory=False.
  pillar_df = pd.read_csv("/data/dzeiberg/pillar_project/dataframe/pillar_data_clinvar38_annotated_final_v13_expanded_031425.csv")


In [3]:
pillar_df.simplified_consequence.value_counts()

simplified_consequence
Missense         864698
Synonymous        75764
Splice Region     49418
Nonsense          35457
Intronic          11252
Unknown            9033
Frameshift         2908
Inframe            1825
5' UTR             1316
3' UTR              813
Start lost          652
Stop lost           409
Other               169
Name: count, dtype: int64

In [4]:
consequence_sets = [
    {'Missense', "Synonymous"},
    set(pillar_df.simplified_consequence.unique()),
    {"Start loss", 'Nonsense', 'Synonymous', 'Missense', 'Frameshift', 'Inframe', 'Stop Loss'}
]
pillar_subsets = [pillar_df[pillar_df.simplified_consequence.isin(consequences)] for consequences in consequence_sets]

In [21]:
consequence_sets

[{'Missense', 'Synonymous'},
 {"3' UTR",
  "5' UTR",
  'Frameshift',
  'Inframe',
  'Intronic',
  'Missense',
  'Nonsense',
  'Other',
  'Splice Region',
  'Start lost',
  'Stop lost',
  'Synonymous',
  'Unknown'},
 {'Frameshift',
  'Inframe',
  'Missense',
  'Nonsense',
  'Start loss',
  'Stop Loss',
  'Synonymous'}]

In [5]:
from pillar_project.data_utils.dataset import Scoreset

In [34]:
scoresets = []
dataset_names = pillar_df["Dataset"].unique()
for pillar_subset in tqdm(pillar_subsets):
    subset_scoresets = []
    with tqdm(dataset_names) as pbar:
        for ds_name in pbar:
            pbar.set_description(ds_name)
            ss = pillar_subset[pillar_subset["Dataset"] == ds_name]
            subset_scoresets.append(Scoreset(ss, quantile_min=0.0, quantile_max=1.0))
    scoresets.append(subset_scoresets)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/86 [00:00<?, ?it/s]

  0%|          | 0/86 [00:00<?, ?it/s]

  0%|          | 0/86 [00:00<?, ?it/s]

In [35]:
counts = pd.DataFrame({(ss.variants[0].Dataset,setID): \
                       {sample_name : int(sample_count) for sample_name,sample_count in zip(ss.sample_names, ss.sample_counts)} \
                        for setID, scoreset_subsets in zip(["Set 1", "Set 2", "Set 3"],scoresets) \
                            for ss in scoreset_subsets} ).T.fillna(0).applymap(int).sort_index(level=0)

/tmp/ipykernel_3141050/1014025806.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  for ss in scoreset_subsets} ).T.fillna(0).applymap(int).sort_index(level=0)


In [36]:
counts

Pathogenic/Likely Pathogenic  \
ASPA_Grønbæk-Thygesen_2024_abundance Set 1                            48   
                                     Set 2                            65   
                                     Set 3                            65   
ASPA_Grønbæk-Thygesen_2024_toxicity  Set 1                            48   
                                     Set 2                            65   
...                                                                  ...   
VHL_Buckley_2024                     Set 2                           198   
                                     Set 3                           169   
XRCC2_unpublished                    Set 1                             0   
                                     Set 2                             2   
                                     Set 3                             0   

                                            Benign/Likely Benign  gnomAD  \
ASPA_Grønbæk-Thygesen_2024_abundance Set 1                     6     338   
                                     Set 2                     6     352   
                                     Set 3                     6     352   
ASPA_Grønbæk-Thygesen_2024_toxicity  Set 1                     6     338   
                                     Set 2                     6     352   
...                                                          ...     ...   
VHL_Buckley_2024                     Set 2                    56     344   
                                     Set 3                     4     269   
XRCC2_unpublished                    Set 1                     1      54   
                                     Set 2                    31     227   
                                     Set 3                     1      57   

                                            Synonymous  
ASPA_Grønbæk-Thygesen_2024_abundance Set 1           0  
                                     Set 2           0  
                                     Set 3           0  
ASPA_Grønbæk-Thygesen_2024_toxicity  Set 1           0  
                                     Set 2           0  
...                                                ...  
VHL_Buckley_2024                     Set 2         375  
                                     Set 3         375  
XRCC2_unpublished                    Set 1          64  
                                     Set 2          64  
                                     Set 3          64  

[258 rows x 4 columns]

In [37]:
counts.to_csv("/data/dzeiberg/pillar_project/dataframe/counts_100_Score_Interval.csv")

In [32]:
pillar_df[(pillar_df.Dataset.str.startswith("BARD1_unpublished")) & \
          (pillar_df.clinvar_sig.isin({"Pathogenic/Likely pathogenic", "Pathogenic", "Likely pathogenic"})) & \
            (pillar_df.simplified_consequence == "Missense")]#.simplified_consequence.value_counts()

,ID,Dataset,Gene,HGNC_id,Chrom,STRAND,hg19_pos,hg38_start,hg38_end,ref_allele,alt_allele,auth_transcript_id,transcript_pos,transcript_ref,transcript_alt,aa_pos,aa_ref,aa_alt,hgvs_c,hgvs_p,consequence,auth_reported_score,auth_reported_rep_score,auth_reported_func_class,auth_reported_normal_min,auth_reported_normal_max,auth_reported_abnormal_min,auth_reported_abnormal_max,splice_measure,gnomad_MAF,clinvar_sig,clinvar_star,clinvar_date_last_reviewed,nucleotide_or_aa,MaveDB URN (score set),Ensembl_transript_ID,Ref_seq_transcript_ID,Model_system,Assay_type,Phenotype_measured,Phenotype_detail,IGVF_produced,simplified_consequence,Flag
937291,BARD1_unpublished_var131539,BARD1_unpublished,BARD1,952.0,2,-1.0,NaN,214797064.0,214797064.0,C,T,NaN,212,G,A,71,C,Y,c.212G>A,p.Cys71Tyr,missense_variant,-0.415165,-4.275768491442718;-6.223014157186438;-1.45960...,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Pathogenic/Likely pathogenic,"criteria provided, multiple submitters, no con...","Jun 09, 2022",nucleotide,NaN,ENST00000260947.9,NM_000465.4,immortalized human cells,Cell viability,Cell survival,overall function,Yes,Missense,NaN


In [38]:
from pathlib import Path
from joblib import dump
savedir = Path("/data/dzeiberg/pillar_project/dataframe/processed_scoresets/")
savedir.mkdir(exist_ok=True)
for setID, scoreset_subsets in tqdm(zip(["Set 1", "Set 2", "Set 3"],scoresets),total=len(scoresets)):
    for ss in tqdm(scoreset_subsets, total=len(scoreset_subsets), leave=False):
        dump(ss, savedir / f"{setID}_{ss.variants[0].Dataset}.pkl")

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/86 [00:00<?, ?it/s]

  0%|          | 0/86 [00:00<?, ?it/s]

  0%|          | 0/86 [00:00<?, ?it/s]